In [1]:
# ZOMATO RECOMMENDATION SYSTEM

In [2]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
#reading the dataset
zomato=pd.read_csv("zomato.csv")


In [6]:
zomato.head() 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [7]:
zomato.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [8]:
zomato.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


In [9]:
zomato.shape

(51717, 17)

In [10]:
zomato.ndim

2

In [12]:
# Remove the nan values from the dataset
zomato.isnull().sum()


url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [14]:
# ?chhanging collumn names

zomato = zomato.rename(columns = {'approx_cost(for two people)' : 'cost', 'listed_in(type)' : 'type', 'listed_in(city)': 'city'})

In [15]:
zomato

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [17]:
zomato_real.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines', 'cost',
       'reviews_list', 'menu_item', 'type', 'city'],
      dtype='object')

In [21]:
zomato_real.duplicated().sum()

0

In [44]:
zomato_1=zomato.drop(['url','dish_liked','phone'],axis=1)

In [45]:
zomato_1

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,Whitefield,Bar,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,Whitefield,Bar,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,Whitefield,Bar,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,"ITPL Main Road, Whitefield",Bar,Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [46]:
zomato_1.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [47]:
zomato_1.duplicated().sum()


43

In [48]:
zomato_1.drop_duplicates(inplace=True)

In [49]:
zomato_1.duplicated().sum()

0

In [50]:
zomato_1.isnull().sum()


address            0
name               0
online_order       0
book_table         0
rate            7767
votes              0
location          21
rest_type        227
cuisines          45
cost             345
reviews_list       0
menu_item          0
type               0
city               0
dtype: int64

In [51]:
zomato_1.dropna(how='any',inplace=True)

In [52]:
zomato_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       43499 non-null  object
 1   name          43499 non-null  object
 2   online_order  43499 non-null  object
 3   book_table    43499 non-null  object
 4   rate          43499 non-null  object
 5   votes         43499 non-null  int64 
 6   location      43499 non-null  object
 7   rest_type     43499 non-null  object
 8   cuisines      43499 non-null  object
 9   cost          43499 non-null  object
 10  reviews_list  43499 non-null  object
 11  menu_item     43499 non-null  object
 12  type          43499 non-null  object
 13  city          43499 non-null  object
dtypes: int64(1), object(13)
memory usage: 5.0+ MB


In [53]:
zomato_1 = zomato_1.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type','listed_in(city)':'city'})

In [54]:
zomato_1.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [55]:
zomatoo['cost'] = zomatoo['cost'].astype(str) #Changing the cost to string
zomatoo['cost'] = zomatoo['cost'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from cost
zomatoo['cost'] = zomatoo['cost'].astype(float) # Changing the cost to Float


In [56]:
zomato_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       43499 non-null  object 
 1   name          43499 non-null  object 
 2   online_order  43499 non-null  object 
 3   book_table    43499 non-null  object 
 4   rate          43499 non-null  object 
 5   votes         43499 non-null  int64  
 6   location      43499 non-null  object 
 7   rest_type     43499 non-null  object 
 8   cuisines      43499 non-null  object 
 9   cost          43499 non-null  float64
 10  reviews_list  43499 non-null  object 
 11  menu_item     43499 non-null  object 
 12  type          43499 non-null  object 
 13  city          43499 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 5.0+ MB


In [57]:
zomato_1['cost']

0         800.0
1         800.0
2         800.0
3         300.0
4         600.0
          ...  
51709     800.0
51711     800.0
51712    1500.0
51715    2500.0
51716    1500.0
Name: cost, Length: 43499, dtype: float64

In [59]:
zomato_1['cost'].sample(20)

8638      300.0
42963     600.0
31168     500.0
31407     400.0
37878     500.0
13543     400.0
37324     300.0
28641     900.0
24475     550.0
7962      250.0
20470     800.0
26715     400.0
33119     300.0
36114     400.0
11000     600.0
46671     200.0
22778    1350.0
41887     400.0
46552     200.0
42818     250.0
Name: cost, dtype: float64

In [61]:
zomato_1['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [66]:
# Remove rows where 'rate' is 'NEW' or '-'
zomato_1 = zomato_1.loc[~zomato_1['rate'].isin(['NEW', '-'])].reset_index(drop=True)

# Define a function to remove '/5' and apply it
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x
zomato_1['rate'] = zomato_1['rate'].apply(remove_slash).str.strip().astype(float)


In [67]:
zomato_1['rate'].head()

0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64

In [68]:
zomato_1['rate'].sample(20)

11821    3.5
20356    3.9
28627    3.0
22561    3.8
19626    3.7
2231     3.8
11459    4.0
7629     3.6
37260    3.3
35966    3.6
14534    2.9
9497     4.3
33279    3.7
24660    3.7
39838    3.8
12464    3.0
13976    3.9
320      3.2
21169    4.6
5287     2.9
Name: rate, dtype: float64

In [69]:
# Adjust the column names
zomato_1.name = zomato_1.name.apply(lambda x:x.title())
zomato_1.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato_1.book_table.replace(('Yes','No'),(True, False),inplace=True)


In [70]:
zomato_1.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [71]:
zomato_1['online_order'].sample(5)

34764    False
5071     False
27817     True
12278    False
40794    False
Name: online_order, dtype: bool

In [72]:
zomato_1['book_table'].sample(5)

3669     False
27277     True
31998    False
35872    False
23416    False
Name: book_table, dtype: bool

In [74]:
zomato_1.cost.unique()

array([ 800.,  300.,  600.,  700.,  550.,  500.,  450.,  650.,  400.,
        900.,  200.,  750.,  150.,  850.,  100., 1200.,  350.,  250.,
        950., 1000., 1500., 1300.,  199., 1100., 1600.,  230.,  130.,
       1700., 1350., 2200., 1400., 2000., 1800., 1900.,  180.,  330.,
       2500., 2100., 3000., 2800., 3400.,   50.,   40., 1250., 3500.,
       4000., 2400., 2600., 1450.,   70., 3200.,  240., 6000., 1050.,
       2300., 4100.,  120., 5000., 3700., 1650., 2700., 4500.,   80.])

In [75]:
zomato_1['city'].unique()

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [76]:
zomato_1.isnull().sum()

address         0
name            0
online_order    0
book_table      0
rate            0
votes           0
location        0
rest_type       0
cuisines        0
cost            0
reviews_list    0
menu_item       0
type            0
city            0
dtype: int64

In [78]:
zomato_1.duplicated().sum()

0

In [79]:
## Computing Mean Rating
restaurants = list(zomato_1['name'].unique())
zomato_1['Mean Rating'] = 0


In [80]:
for i in range(len(restaurants)):
    zomato_1['Mean Rating'][zomato_1['name'] == restaurants[i]] = zomato_1['rate'][zomato_1['name'] == restaurants[i]].mean()

In [81]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [83]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (1,5))

zomato_1[['Mean Rating']] = scaler.fit_transform(zomatoo[['Mean Rating']]).round(2)

In [85]:
zomato_1.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
26528,"12/13, 20th Main, Maruthinagar, 1st Stage",Neer Dosa Fish Land,False,False,3.6,39,BTM,Casual Dining,"Seafood, Mangalorean",600.0,"[('Rated 4.0', 'RATED\n Great Food. Can be Aw...",[],Delivery,Koramangala 6th Block,3.32
31149,"50, Little Plaza, Cunningham Road",Chels Cafe,True,False,3.5,20,Cunningham Road,Cafe,"Cafe, Burger",500.0,"[('Rated 4.0', 'RATED\n Visited this place wi...","['Chicken Crispy Patty Burger', 'Veg Crispy Pa...",Dine-out,Lavelle Road,3.19
31800,"33, 5th Main, Sheshadri Road, Near Ananda Rao ...",Shaken & Stirred,True,False,3.9,86,Race Course Road,"Bar, Casual Dining","Continental, North Indian, Chinese",1500.0,"[('Rated 3.0', ""RATED\n Went there for a quic...",[],Delivery,Malleshwaram,3.71
39534,"Opposite To South Indian Bank, Near Carmelram ...",Sana'S Restaurant,True,False,3.3,19,Sarjapur Road,Quick Bites,"Biryani, Chettinad, South Indian, Chinese",600.0,"[('Rated 5.0', 'RATED\n Thnk. U for exception...",[],Delivery,Sarjapur Road,2.94
24618,"First A Cross, Jyoti Nivas College Road, KHB C...",Meenu Oota Da Maane,True,True,4.2,296,Koramangala 5th Block,Casual Dining,"Seafood, Mangalorean, Chinese",800.0,"[('Rated 5.0', ""RATED\n For the best Manglore...",[],Dine-out,Koramangala 5th Block,3.35


In [86]:
## Text Preprocessing

# Lower casing
# Removal of Punctuations
# Removal of Stopwords
# Removal of URLs
# Spelling correction

In [87]:
zomato_1["reviews_list"] = zomato_1["reviews_list"].str.lower()
zomato_1["cuisines"] = zomato_1["cuisines"].str.lower()


In [89]:
zomato_1[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
2818,"[('rated 4.0', ""rated\n place was kinda ok.. ...","north indian, chinese"
26349,"[('rated 3.5', 'rated\n i love the food, but ...","arabian, biryani, north indian, chinese, fast ..."
30767,"[('rated 1.0', 'rated\n is this place really ...","north indian, lucknowi"
20066,"[('rated 4.0', 'rated\n it was a casual weeke...","desserts, cafe, italian, beverages"
31973,"[('rated 4.0', 'rated\n i ordered masala dosh...","south indian, north indian"


In [90]:
import string

In [91]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato_1["reviews_list"] = zomato_1["reviews_list"].apply(lambda text: remove_punctuation(text))


In [92]:
zomato_1[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
17618,rated 10 ratedn ordered rajbhog kulfi family ...,desserts
34806,rated 50 ratedn best class sweets better to o...,"street food, mithai"
26174,rated 50 ratedn iãx83ãx83ãx82ãx83ãx83ãx...,"continental, fast food, italian, north indian"
9053,rated 40 ratedn okay ill be honest i did not ...,"chinese, north indian"
19674,rated 30 ratedn its was a hookah parlour now ...,"north indian, chinese"


In [93]:
# stopwords removal
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato_1["reviews_list"] = zomato_1["reviews_list"].apply(lambda text: remove_stopwords(text))

In [95]:
zomato_1["reviews_list"][5]

'rated 30 ratedn food 35nambience 35nservice 35nnhad family lunch place serves buffets ordered soups babycorn starters butter naan kadai panner veg kohlapuri pease pulav food good service slow make wait 20 mins get order place apt family hangoutnncheers'

In [96]:
# Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [97]:
zomato_1["reviews_list"] = zomato_1["reviews_list"].apply(lambda text: remove_urls(text))

In [103]:
zomato_1["reviews_list"][1]

'rated 40 ratedn dinner family turned good choose suitable ages people try place liked starters service good prices affordable recommend restaurant early dinner place little noisy rated 30 ratedn ambience really nice staff courteous price pretty high quantity overall experience fine quality food nice nothing extraordinary also buffetonly veg rated 30 ratedn felt good little expensive quantity serve terms taste decent nothing much talk ambience regular casual dining restaurant take family dinner lunch improve quantity may reduce price bit may improve presentation food might manage get repeat customers rated 40 ratedn looking quite place spend time family well wanted try new place since banashankari thought trying place place good rating part zomato gold decided try place delite see friendly staff food ordered tasty wellnnfood 45nambience 35nfriendly staff 45npocket friendly 45nnwill definitely visit rated 40 ratedn nice place dine good ambiance food good serving time also goodneat restr

In [104]:
restaurant_names = list(zomato_1['name'].unique())

In [105]:
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [106]:
def get_top_words(column, top_nu_of_words, nu_of_word):
    
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    
    bag_of_words = vec.fit_transform(column)
    
    sum_words = bag_of_words.sum(axis=0)
    
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:top_nu_of_words]

In [108]:
zomato_1.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
18137,"Eden Park Restaurants, 722/2, 36th Cross, 10th...",Divina - Eden Park Restaurants,False,False,4.0,147,Jayanagar,Dessert Parlor,desserts,400.0,rated 30 ratedn sunday noonnunfortunately ever...,[],Delivery,JP Nagar,3.84
13892,"6th Cross Road, HAL 2nd Stage, Old Airport Roa...",Shanmukha,True,False,3.8,138,Old Airport Road,Casual Dining,"biryani, andhra, north indian, chinese",800.0,rated 10 ratedn ordered chicken biryani shanmu...,[],Delivery,Indiranagar,3.81
439,"186, 11th Main, 4th Cross, Hanumanthnagar, Ban...",Sugar Rush,False,False,3.8,39,Banashankari,Bakery,"bakery, desserts",300.0,rated 10 ratedn wanted try cupcakes could reac...,[],Desserts,Banashankari,3.58
632,"68, Opposite ICICI Bank, 21st Main Road, 2nd S...",Parjanya Chat Zone,False,False,3.3,17,Banashankari,Quick Bites,"street food, fast food",200.0,rated 10 ratedn cant go lower thisnone dirties...,[],Dine-out,Banashankari,2.94
40296,"122/3, Sai Kamala Complex, ITPL Main Road, Bro...",Le Charcoal,True,True,4.1,345,Brookefield,Casual Dining,"arabian, lebanese, mughlai, north indian, bbq,...",600.0,rated 50 ratedn good experience le charcoalnfo...,[],Delivery,Whitefield,3.86


In [109]:
zomato_1=zomato_1.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [110]:
zomato_1.head()

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,Banashankari,"north indian, mughlai, chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99
1,Spice Elephant,True,False,4.1,Banashankari,"chinese, north indian, thai",800.0,rated 40 ratedn dinner family turned good choo...,Banashankari,3.97
2,San Churro Cafe,True,False,3.8,Banashankari,"cafe, mexican, italian",800.0,rated 30 ratedn ambience good enough pocket fr...,Banashankari,3.58
3,Addhuri Udupi Bhojana,False,False,3.7,Banashankari,"south indian, north indian",300.0,rated 40 ratedn great food proper karnataka st...,Banashankari,3.45
4,Grand Village,False,False,3.8,Basavanagudi,"north indian, rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,3.58


In [111]:
# Randomly sample 60% of your dataframe
df_percent = zomato_1.sample(frac=0.5)

In [112]:
df_percent.shape

(20618, 10)

In [113]:
df_percent.set_index('name', inplace=True)

In [114]:
indices = pd.Series(df_percent.index)

In [115]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

In [116]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [117]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top 10 restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new

In [118]:
df_percent[df_percent.index == 'Pai Vihar'].head()

,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,,,
Pai Vihar,False,False,3.2,City Market,"south indian, street food, chinese, fast food",400.0,rated 20 ratedn food dry bland dont understand...,Church Street,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 20 ratedn nice place vasanthnagar extrem...,Brigade Road,2.48
Pai Vihar,False,False,3.2,City Market,"south indian, street food, chinese, fast food",400.0,rated 20 ratedn food dry bland dont understand...,Brigade Road,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,Lavelle Road,2.48
Pai Vihar,True,False,2.8,Vasanth Nagar,"south indian, street food, chinese, fast food",400.0,rated 30 ratedn 12 rate herenneven though tast...,Frazer Town,2.48


In [119]:

recommend('Pai Vihar')

TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Gokul Kuteera,"north indian, chinese",3.71,650.0
Cinnamon,"north indian, asian, continental",3.62,1000.0
Desi Doze,"north indian, fast food",3.58,400.0
Ghar Ka Khana,north indian,3.45,150.0
Agarwal Food Service,"north indian, chinese, biryani",3.39,400.0
Sri Krishna Kitchen,fast food,3.34,400.0
Mayura Sagar,"chinese, north indian, south indian",3.32,250.0
Food Point,north indian,2.94,450.0
Punjabi Tasty Khana,"north indian, chinese, biryani",2.68,450.0
Punjabi Dawat,"north indian, chinese",2.42,400.0


In [120]:
recommend('Jalsa')

TOP 10 RESTAURANTS LIKE Jalsa WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Asia Kitchen By Mainland China,"asian, chinese, thai, momos",5.00,1500.0
Big Pitcher,"american, continental, north indian, mediterra...",4.68,1800.0
Communiti,"continental, bbq, salad",4.67,1500.0
Hammered,"north indian, thai, japanese, continental, cafe",4.65,1300.0
Brew And Barbeque - A Microbrewery Pub,"continental, north indian, bbq, steak",4.64,1400.0
The Globe Grub,"continental, north indian, asian, italian",4.48,1300.0
1131 Bar + Kitchen,"continental, asian, italian, north indian",4.48,1500.0
Brooks And Bonds Brewery,"continental, mediterranean, north indian, chin...",4.45,1600.0
Deja Vu Resto Bar,"north indian, italian",4.35,900.0
The Fisherman'S Wharf,"seafood, goan, north indian, continental, asian",4.30,1400.0


In [121]:
recommend('Timepass Dinner')

TOP 7 RESTAURANTS LIKE Timepass Dinner WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Roots,"north indian, south indian, chinese, continent...",4.53,1200.0
Gokul Kuteera,"north indian, chinese, south indian",3.71,650.0
Atithi,"north indian, chinese, street food",3.63,800.0
Cinnamon,"north indian, asian, continental",3.62,1000.0
7 Days Restaurant,"north indian, chinese, arabian",3.58,400.0
Chef In,"biryani, north indian, chinese",3.32,500.0
Zaika Take Away,"north indian, chinese, thai",1.77,500.0
